In [8]:
import json

path = "bigdata-exam-environment/Files/movies.jsonl"
movies_df = spark.read.json(path).cache()

The type of our dataset object is DataFrame

In [18]:
type(movies_df)

pyspark.sql.dataframe.DataFrame

Print the schema

In [19]:
movies_df.printSchema()

root
 |-- actors: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- birth: string (nullable = true)
 |    |    |-- death: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- name: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- votes: string (nullable = true)
 |-- year: string (nullable = true)



In [46]:
from pyspark.sql.functions import explode, avg, split
from pyspark.sql.types import IntegerType, NumericType, decimal, DecimalType, DoubleType
from pyspark.sql.functions import split
from pyspark.sql.functions import col

In [63]:
data = movies_df.select(explode("actors").alias("i"), "i.name", "i.birth", "year")

In [66]:
datainteger = data.select(data["year"].cast(IntegerType()), data["birth"].cast(IntegerType()), "name")

In [82]:
minage = datainteger.withColumn("age",  datainteger["year"] - datainteger["birth"]).groupBy("name", "year").min("age")

In [89]:
minage.filter(minage["min(age)"] == 15).orderBy("year", ascending = False ).show()

+------------------+----+--------+
|              name|year|min(age)|
+------------------+----+--------+
|       Corbyn Lowe|2013|      15|
|       Fynn Henkel|2011|      15|
|   Jonathan Dümcke|2006|      15|
|       Nick Fowler|2006|      15|
|         Neil Hope|1987|      15|
|      Gary Coleman|1983|      15|
|     Philip McKeon|1979|      15|
|Sergei Krupennikov|1978|      15|
|  Dmitri Krechetov|1975|      15|
|    Shawn Campbell|1973|      15|
|     Roderick Shaw|1973|      15|
|  Simon Gipps-Kent|1973|      15|
|     Keith Chegwin|1972|      15|
|Manuel Padilla Jr.|1970|      15|
|      Milos Vognic|1969|      15|
| John David Carson|1967|      15|
|Balázs Kosztolányi|1963|      15|
|       Rusty Hamer|1962|      15|
|        Tim Rooney|1962|      15|
|     Gilles Payant|1962|      15|
+------------------+----+--------+
only showing top 20 rows



In [37]:
movies_df.filter(movies_df["name"] == "Mob Town").select(explode(movies_df["actors"]).alias("i"), "i.birth", movies_df["year"].cast(IntegerType())).show()

+--------------------+-----+----+
|                   i|birth|year|
+--------------------+-----+----+
|{1919, 1988, Gabr...| 1919|1941|
|{1910, 1979, Dick...| 1910|1941|
|{1920, 1976, Bill...| 1920|1941|
|{1920, 1999, Hunt...| 1920|1941|
+--------------------+-----+----+



In [41]:
movies_df.select(explode("actors").alias("i"), "i.name").groupBy("i.name").count().sort("count", ascending = False).show()

+--------------+-----+
|          name|count|
+--------------+-----+
|     Mel Blanc| 1186|
|   Jack Mercer|  635|
|Michael Landon|  591|
|  James Arness|  578|
| Milburn Stone|  577|
|  Raymond Burr|  499|
|   Daws Butler|  493|
|    Paul Frees|  385|
|  Lorne Greene|  380|
|   Dan Blocker|  364|
|  Jackson Beck|  359|
|   Don Messick|  359|
|Richard Crenna|  358|
|Edgar Buchanan|  358|
|Walter Brennan|  345|
|  Peter Thomas|  334|
| Dennis Weaver|  320|
|  Ozzie Nelson|  318|
|  Ricky Nelson|  315|
|  David Nelson|  313|
+--------------+-----+
only showing top 20 rows



In [53]:
movies_df.select("year", movies_df["votes"].cast(IntegerType())).groupBy("year").sum("votes").orderBy("sum(votes)", ascending = False).show()

+----+----------+
|year|sum(votes)|
+----+----------+
|1957|   1673009|
|1954|   1586085|
|1960|   1539885|
|1959|   1525685|
|1962|   1372253|
|1968|   1239030|
|1939|   1165867|
|1964|   1138673|
|1955|   1128751|
|1958|   1094661|
|1941|   1081774|
|1953|   1081183|
|1963|   1074507|
|1950|   1043050|
|1946|   1014856|
|1961|    998506|
|1940|    978235|
|1942|    972899|
|1951|    931534|
|1956|    872253|
+----+----------+
only showing top 20 rows



In [19]:
movies_df.select(explode("actors").alias("i"), "i.name", movies_df["rating"].cast(DoubleType())).groupBy("i.name").avg("rating").orderBy("avg(rating)", ascending = False).show()

+--------------------+-----------------+
|                name|      avg(rating)|
+--------------------+-----------------+
|       John H. Moore|             10.0|
|      Charles Cullum|             10.0|
|     Walter Kiaulehn|             10.0|
|     Willem Holsboer|             10.0|
|    Kurt Wehofschitz|              9.9|
|    Erwin Scherschel|              9.8|
|       Ottokar Runze|              9.8|
|         John Wesley|              9.7|
|        Eugen Verber|              9.7|
|          Karl Luley|              9.6|
|      Walter Tarrach|              9.6|
|       Allen Sferios|              9.6|
|   Friedrich Siemers|              9.6|
|     Herbert Johnson|              9.6|
|Hans Müller-Weste...|              9.6|
|         Otto Collin|              9.6|
|        Paul Günther|              9.6|
|     Bernd M. Bausch|              9.6|
|      Wolfgang Spier|             9.55|
|     Beppo Schwaiger|9.533333333333333|
+--------------------+-----------------+
only showing top

In [55]:
average_rating= movies_df.select(avg(movies_df["rating"].cast(DoubleType()))).collect()[0][0]

In [56]:
average_rating

6.673361642793781

In [57]:
Genre= movies_df.select.select(movies_df["rating"].cast(DoubleType()), "genres").groupby("genres").avg("rating")

In [61]:
Genre.show()

+--------------------+------------------+
|              genres|       avg(rating)|
+--------------------+------------------+
|Action,Adventure,...| 5.771428571428572|
|        Comedy,Sport| 6.043181818181819|
| Adult,Horror,Sci-Fi|               6.9|
|  Action,War,Western|              5.75|
|  Crime,Horror,Short|               6.9|
|Drama,Romance,Tal...|               5.3|
|  Action,Adult,Short|               4.5|
|Fantasy,Horror,Mu...|               7.6|
| Music,Musical,Short|              6.85|
|Adventure,Family,...| 6.924489795918366|
|Comedy,Drama,Western| 6.288461538461538|
| Documentary,Western|6.8999999999999995|
|Game-Show,Reality-TV|              8.25|
|Comedy,Family,His...|               7.0|
|Film-Noir,Horror,...|              6.35|
|   Music,Short,Sport|               5.9|
|Fantasy,Mystery,T...|               7.1|
|Fantasy,Sci-Fi,Th...|               6.6|
|   Documentary,Sport| 6.915789473684211|
|Comedy,Musical,Sc...|             5.325|
+--------------------+------------

In [60]:
Genre.filter(Genre["avg(rating)"] > average_rating).show()

+--------------------+------------------+
|              genres|       avg(rating)|
+--------------------+------------------+
| Adult,Horror,Sci-Fi|               6.9|
|  Crime,Horror,Short|               6.9|
|Fantasy,Horror,Mu...|               7.6|
| Music,Musical,Short|              6.85|
|Adventure,Family,...| 6.924489795918366|
| Documentary,Western|6.8999999999999995|
|Game-Show,Reality-TV|              8.25|
|Comedy,Family,His...|               7.0|
|Fantasy,Mystery,T...|               7.1|
|   Documentary,Sport| 6.915789473684211|
|Action,Animation,...| 7.066666666666666|
|Film-Noir,Mystery...| 7.166666666666667|
|Documentary,Horro...|               6.8|
|       Comedy,Family| 7.719479218828239|
|    Animation,Comedy| 7.154166666666669|
|Biography,Documen...| 6.940909090909091|
|Drama,Fantasy,Fil...|               7.2|
|               Crime| 6.713854351687388|
|Biography,Drama,M...| 6.736363636363635|
|Musical,Romance,T...|              6.85|
+--------------------+------------

In [27]:
movies_df.select(movies_df["rating"].cast(IntegerType()), movies_df["votes"].cast(IntegerType())).groupby("rating").avg("votes").show()

+------+------------------+
|rating|        avg(votes)|
+------+------------------+
|     1|            449.72|
|     6|294.60998965794613|
|     3|154.43396226415095|
|     5|176.38442840306422|
|     9|165.26829268292684|
|     4|124.35920889987639|
|     8|1788.9719594594594|
|     7| 763.4944402324994|
|    10|               5.5|
|     2|  168.562874251497|
+------+------------------+



In [23]:
movies_df.select(explode("actors").alias("i"), "i.name").select("name").distinct().count()

22737

In [83]:
movie_actor = movies_df.select(explode("actors").alias("i"), "i.name", "name")
movie_actor.filter(movie_actor["i.name"] == "Mickey Rooney").count()

118

In [85]:
movies_df.select(explode("actors").alias("i"), "i.name", movies_df["rating"].cast(IntegerType())).groupBy("i.name").avg("rating").orderBy(["avg(rating)", "name"], ascending = False).show()

+-----------------+-----------+
|             name|avg(rating)|
+-----------------+-----------+
|  Willem Holsboer|       10.0|
|  Walter Kiaulehn|       10.0|
|    John H. Moore|       10.0|
|   Charles Cullum|       10.0|
|     Zorko Rajcic|        9.0|
|     Zale Kessler|        9.0|
|   Wolfgang Spier|        9.0|
|   Willi Hoffmann|        9.0|
|Werner Hessenland|        9.0|
|     Werner Dahms|        9.0|
|   Walter Tarrach|        9.0|
|      Walter Hoor|        9.0|
|   Vukan Dimevski|        9.0|
|      Tommy Vance|        9.0|
|       Tom Snyder|        9.0|
|       Todd Davis|        9.0|
|       Tito Vuolo|        9.0|
|    Timmy Everett|        9.0|
|        Tim Meats|        9.0|
|     Tibor Badari|        9.0|
+-----------------+-----------+
only showing top 20 rows



Print one row

In [12]:
orders_df.limit(1).collect()

[Row(customer=Row(first_name='Preston', last_name='Landry'), date='2018-2-4', items=[Row(price=1.53, product='fan', quantity=5), Row(price=1.33, product='computer screen', quantity=6), Row(price=1.06, product='kettle', quantity=6), Row(price=1.96, product='stuffed animal', quantity=3), Row(price=1.09, product='the book', quantity=7), Row(price=1.42, product='headphones', quantity=9), Row(price=1.67, product='whiskey bottle', quantity=3)], order_id=0)]

You can access the underlying RDD object and use any functions you learned for Spark RDDs.

In [13]:
orders_df.rdd.filter(lambda ordr: ordr.customer.last_name == "Landry").count()

1960

In [100]:
movies_df_interger = movies_df.select(split("genres", ",",3).alias("genre"), movies_df["rating"].cast(IntegerType()))

In [117]:
movies_df_interger.select(explode("i").alias("genre"), "rating").groupBy("genre").avg("rating").show()

+-----------+------------------+
|      genre|       avg(rating)|
+-----------+------------------+
|      Crime| 6.402219922867086|
|    Romance| 5.946172712017555|
|   Thriller| 5.952978056426332|
|  Adventure| 5.994167735915083|
|      Drama| 6.244049980166601|
|        War| 6.318181818181818|
|Documentary| 6.527777777777778|
| Reality-TV| 6.304347826086956|
|     Family| 6.673382705292965|
|    Fantasy|   6.5446735395189|
|  Game-Show| 5.904761904761905|
|      Adult| 5.486725663716814|
|    History| 6.352576039726878|
|    Mystery| 6.426814268142682|
|    Musical| 5.833333333333333|
|  Animation|   6.1298755186722|
|      Music| 6.115518096182449|
|  Film-Noir|6.1834239130434785|
|      Short| 5.730222613507735|
|     Horror| 5.859930615784909|
+-----------+------------------+
only showing top 20 rows



In [111]:
avg _df_interger.groupBy().avg("rating").collect()[0][0]

6.229230574217264

In [63]:
movies_df.select(split("genres", ","), "rating").(explode("split(genres, ,, -1)").alias("genre"), movies_df["rating"]).show()

+--------------------+------+
|split(genres, ,, -1)|rating|
+--------------------+------+
|          [Thriller]|   3.8|
|[Action, Adventur...|   7.9|
|[Comedy, Crime, D...|   7.9|
|[Animation, Famil...|   6.2|
|            [Comedy]|   7.1|
|         [Adventure]|   5.8|
|[Drama, Family, R...|   8.9|
|             [Drama]|   6.1|
|[Adventure, Comed...|   7.6|
|[Action, Drama, R...|   5.4|
|[Animation, Comed...|   7.1|
|    [Drama, Romance]|   5.7|
|   [Comedy, Musical]|   5.3|
|            [Comedy]|   7.4|
|            [Comedy]|   4.5|
|            [Comedy]|   6.4|
|[Adventure, Crime...|   5.2|
|[Documentary, Short]|   6.3|
|[Action, Adventur...|   7.8|
|  [Animation, Short]|   6.9|
+--------------------+------+
only showing top 20 rows



In [114]:
avg = movies_df_interger.groupBy().avg("rating").collect()[0][0]

In [14]:
from pyspark.sql.functions import array_contains

In [56]:
movie_mick = movies_df.filter(array_contains(movies_df["actors"]["name"], "Mickey Rooney"))

In [57]:
movie_mick.filter(movie_mick["name"] != "Mickey Rooney").select("name").distinct().count()

118

In [31]:
from pyspark.sql.functions import length

In [46]:
movies_df.filter(movies_df["name"] == "Desperate Trails").select(explode("actors")).show()

+--------------------+
|                 col|
+--------------------+
|{1880, 1954, Edwa...|
|{1878, 1947, Harr...|
|{1909, 2010, Geor...|
|{1877, 1959, Russ...|
|{1904, 1974, John...|
|{1910, 1975, Bob ...|
|{1901, 1976, Fuzz...|
+--------------------+



In [49]:
movienn = movies_df.select(explode("actors").alias("i"), "i.name", "name")

In [53]:
movienn.groupBy(movies_df["name"]).count().orderBy("count", ascending = False).show()

+--------------------+-----+
|                name|count|
+--------------------+-----+
|        Episode #1.1|   78|
|        Episode #1.2|   63|
|               Pilot|   63|
|            The Trap|   56|
|        Episode #1.3|   46|
|        The Stranger|   44|
|         The Hostage|   39|
|        The Fugitive|   39|
|Where There's a Will|   38|
|         The Witness|   36|
|      Double Trouble|   34|
|             Othello|   34|
|              Hamlet|   32|
|        Episode #1.4|   31|
|          The Search|   31|
|The Three Musketeers|   31|
|            The Hero|   31|
|        The Prisoner|   30|
|          Homecoming|   30|
|             The Gun|   29|
+--------------------+-----+
only showing top 20 rows



In [59]:
micandwynn = movies_df.filter(array_contains(movies_df["actors"]["name"], "Mickey Rooney")).filter(array_contains(movies_df["actors"]["name"], "Keenan Wynn"))

In [60]:
micandwynn.select("name").distinct().count()

2

### 1.2. Dataframe Operations
We perform some queries using operations on Dataframes ([Here](https://spark.apache.org/docs/2.3.0/sql-programming-guide.html#untyped-dataset-operations-aka-dataframe-operations) is a guide on DF Operations with a link to the [API Documentation](https://spark.apache.org/docs/2.3.0/api/python/pyspark.sql.html))

We can select columns and show the result

In [25]:
orders_df.select("customer.first_name", "customer.last_name").limit(5).show()

+----------+---------+
|first_name|last_name|
+----------+---------+
|   Preston|   Landry|
|    Jamari|Dominguez|
|   Brendon|  Sicilia|
|    Armani|   Ardeni|
|    Jamari|     Miao|
+----------+---------+



As you can see we can navigate to the nested items with the dot

In [26]:
orders_df.filter(orders_df["customer.last_name"] == "Landry").count()

1960

How about nested arrays?

In [14]:
orders_df.select("order_id", "items").orderBy("order_id").limit(5).show()

+--------+--------------------+
|order_id|               items|
+--------+--------------------+
|       0|[{1.53, fan, 5}, ...|
|       1|[{1.61, fan, 7}, ...|
|       2|[{1.41, the book,...|
|       3|[{1.05, computer ...|
|       4|[{1.92, headphone...|
+--------+--------------------+



Let us try to find orders of a fan.

In [12]:
orders_df.filter(array_contains(orders_df["items.product"], "fan")).show()

+--------------------+--------+--------------------+--------+
|            customer|    date|               items|order_id|
+--------------------+--------+--------------------+--------+
|   {Preston, Landry}|2018-2-4|[{1.53, fan, 5}, ...|       0|
| {Jamari, Dominguez}|2016-1-8|[{1.61, fan, 7}, ...|       1|
|  {Brendon, Sicilia}|2016-6-6|[{1.41, the book,...|       2|
|      {Jamari, Miao}|2016-4-6|[{1.92, headphone...|       4|
|   {Jensen, Balster}|2017-4-9|[{1.87, fan, 5}, ...|       5|
|    {Carlo, Marinko}|2017-1-3|[{1.65, whiskey b...|       8|
|      {Ashtyn, Dahl}|2016-2-7|[{1.85, whiskey b...|       9|
|  {Aydin, Mc namara}|2018-3-4|[{1.35, kettle, 7...|      12|
|      {Briana, Egan}|2018-6-1|[{1.19, toaster, ...|      14|
|  {Niko, Berenguier}|2016-1-1|[{1.15, fan, 10},...|      16|
|{Kristen, Bridgeman}|2018-3-4|[{1.92, mouse tra...|      17|
|      {Averi, Drago}|2018-4-4|[{1.3, whiskey bo...|      18|
|     {Kendra, Horah}|2017-3-3|[{1.65, notebook,...|      21|
|{Janiya

In [17]:
orders_df.filter(array_contains("items.product", "fan")).count()

32778

The above code doesn't work! Use ```array contains``` instead.

In [18]:
from pyspark.sql.functions import array_contains

orders_df.filter(array_contains("items.product", "fan")).count()

32778

Let us try to unnest the data.

Unnest the products with explode.

Explode will generate as many rows as there are elements in the array and match them to other attributes.

In [15]:
from pyspark.sql.functions import explode

orders_df.select(explode("items").alias("i"), "i.price", "order_id").orderBy("order_id").limit(5).show()

+--------------------+-----+--------+
|                   i|price|order_id|
+--------------------+-----+--------+
|      {1.53, fan, 5}| 1.53|       0|
|{1.33, computer s...| 1.33|       0|
|   {1.06, kettle, 6}| 1.06|       0|
|{1.96, stuffed an...| 1.96|       0|
| {1.09, the book, 7}| 1.09|       0|
+--------------------+-----+--------+



In [22]:
orders_df.select(explode("items").alias("i"), "i.product", "order_id").orderBy("order_id", ascending = False).show()

+--------------------+---------------+--------+
|                   i|        product|order_id|
+--------------------+---------------+--------+
|{1.91, whiskey bo...| whiskey bottle|   99999|
|{1.57, mouse trap...|     mouse trap|   99999|
|  {1.02, toaster, 8}|        toaster|   99999|
|  {1.81, toaster, 7}|        toaster|   99999|
|{1.76, computer s...|computer screen|   99999|
|{1.55, headphones...|     headphones|   99998|
|      {1.16, fan, 7}|            fan|   99998|
|{1.27, headphones...|     headphones|   99998|
|   {1.26, kettle, 7}|         kettle|   99998|
| {1.4, the book, 10}|       the book|   99998|
|{1.62, whiskey bo...| whiskey bottle|   99997|
|  {1.37, toaster, 2}|        toaster|   99997|
| {1.73, notebook, 8}|       notebook|   99997|
|{1.1, headphones, 2}|     headphones|   99997|
|  {1.92, toaster, 4}|        toaster|   99997|
|{1.03, computer s...|computer screen|   99996|
|{1.7, stuffed ani...| stuffed animal|   99996|
|      {1.99, fan, 7}|            fan|  

Now we can use this table to filter.

In [37]:
exploded_df = orders_df.select(explode("items").alias("i"), "i.product", "order_id")

In [39]:
exploded_df.filter(exploded_df["product"] == "fan").limit(5).show()

+--------------+-------+--------+
|             i|product|order_id|
+--------------+-------+--------+
|{1.53, fan, 5}|    fan|       0|
|{1.61, fan, 7}|    fan|       1|
| {1.1, fan, 7}|    fan|       2|
|{1.15, fan, 8}|    fan|       2|
|{1.44, fan, 2}|    fan|       4|
+--------------+-------+--------+



In [ ]:
exploded_df = orders_df.select(explode("items").alias("i"), "i.product", "order_id")
exploded_df.filter(exploded_df["product"] == "fan").count()

You might have tried to access the i.product column directly using a ```.filter``` right after the ```.select```. That, however, does not work, because the column is not available to ```orders_df``` when creating a clause like ```(orders_df["i.product"] == "fan")```. A possible workaround when using Dataframe operations is that of using a string clause in ```.filter```, so that the product column will be resolved after it has been added with the ```.select```.

In [ ]:
orders_df.select(explode("items").alias("i"), "i.product", "order_id").filter("product = 'fan'").count()

Project the nested columns

In [42]:
orders_df.select(explode("items").alias("i"), "*").select("order_id", "i.*", "customer.*").limit(5).show()

+--------+-----+---------------+--------+----------+---------+
|order_id|price|        product|quantity|first_name|last_name|
+--------+-----+---------------+--------+----------+---------+
|       0| 1.53|            fan|       5|   Preston|   Landry|
|       0| 1.33|computer screen|       6|   Preston|   Landry|
|       0| 1.06|         kettle|       6|   Preston|   Landry|
|       0| 1.96| stuffed animal|       3|   Preston|   Landry|
|       0| 1.09|       the book|       7|   Preston|   Landry|
+--------+-----+---------------+--------+----------+---------+



In [ ]:
orders_df.select(explode("items").alias("i"), "*").select(
    "order_id", "customer.*", "date", "i.*").limit(3).show()

In [ ]:
movies_df.select(explode("="))

### 1.3 Exercises

1) Find the average quantity at which each product is purchased. Only show the top 10 products by quantity. (Hint: you may need to import the function ```desc``` from ```pyspark.sql.functions``` to define descending order)

In [51]:
exploded_table =orders_df.select(explode("items").alias("i"), "*").select("i.*", "customer.*", "order_id")

In [52]:
exploded_table.limit(5).show()

+-----+---------------+--------+----------+---------+--------+
|price|        product|quantity|first_name|last_name|order_id|
+-----+---------------+--------+----------+---------+--------+
| 1.53|            fan|       5|   Preston|   Landry|       0|
| 1.33|computer screen|       6|   Preston|   Landry|       0|
| 1.06|         kettle|       6|   Preston|   Landry|       0|
| 1.96| stuffed animal|       3|   Preston|   Landry|       0|
| 1.09|       the book|       7|   Preston|   Landry|       0|
+-----+---------------+--------+----------+---------+--------+



In [53]:
exploded_table.groupBy("product").avg("quantity").limit(100).show()

+---------------+-----------------+
|        product|    avg(quantity)|
+---------------+-----------------+
|       the book|5.514178678641427|
|     mouse trap|5.503895651308093|
|computer screen|5.504839685420448|
| whiskey bottle|5.475555222463714|
|        toaster|5.515549016184942|
| stuffed animal|5.470854598218753|
|         kettle|5.512053325314489|
|            fan|5.496342868593758|
|     headphones|5.485920795060985|
|       notebook|5.483182341458532|
+---------------+-----------------+



2) Find the most expensive order

In [56]:
exploded_table.groupBy("order_id").sum("price").orderBy("sum(price)", ascending = False).limit(5).show()

+--------+------------------+
|order_id|        sum(price)|
+--------+------------------+
|   98235|             13.21|
|   36484|13.170000000000002|
|   11409|             12.97|
|   74460|12.870000000000001|
|    9913|             12.86|
+--------+------------------+



## 2. Spark SQL

Spark SQL allows the users to formulate their queries using SQL. The requirement is the use of Dataframes, which as said before are similar to relational tables. In addition to a familiar interface, writing queries in SQL might provide better performance than RDDs, inheriting efficiency from the Dataframe operations, while also performing automatic optimization of queries.

But what if we are interested not only in the number of dates, but the actual
dates themselves? Luckily Spark Dataframes / Spark SQL do provide us with methods to preserve the original information of the date list. If now we would like to know for each customer, on which dates they placed an order, we shall use <font face="courier">collect_set</font> method:

In [10]:
from pyspark.sql.functions import collect_set
orders_df.groupBy("customer.first_name", "customer.last_name").agg(collect_set("date")).show()

+----------+--------------------+--------------------+
|first_name|           last_name|   collect_set(date)|
+----------+--------------------+--------------------+
|     Abbie|                Egan|[2018-4-8, 2016-3...|
|  Abigayle|           Mc namara|[2016-4-2, 2017-6-9]|
|   Adalynn|              Ardeni|[2018-2-2, 2018-6...|
|      Aden|          Rosenbloom|         [2016-3-10]|
|    Adonis|              Badash|          [2017-6-8]|
|   Agustin|          Srivastava|         [2018-4-10]|
|     Aiden|             Suchoff|          [2018-2-8]|
|    Aiyana|              Landry|          [2018-2-3]|
|    Alaina|              Gruber|[2016-3-1, 2018-5-1]|
|    Alayna|               Mayer|         [2016-3-10]|
|Alexandria|         Butterfield|[2018-5-3, 2017-4...|
|Alexzander|              Landry|[2017-1-3, 2017-6-3]|
|     Alice|             Balster|[2018-4-4, 2017-4-7]|
|     Allan|                  Po|[2016-1-9, 2016-5...|
|     Allie|          Berenguier|[2016-2-1, 2017-1...|
|    Alyvi

In [ ]:
%%sql
select customer.first_name, customer.last_name, collect_set("date") from orders group by customer.first_name, customer.last_name

For some other cases where we want to preserve all the entries rather than the de-duplicated ones, we can use  <font face="courier">collect_list</font> method. For example, for each date we want to record the first and last names of customers. Since two customers might share the same last name, we need to collect all of them. The query should look like this:

In [11]:
from pyspark.sql.functions import collect_list
orders_df.groupBy("date").agg(collect_list("customer.last_name")).show()

+---------+--------------------------------+
|     date|collect_list(customer.last_name)|
+---------+--------------------------------+
| 2017-4-8|            [Egan, Gruber, Le...|
| 2016-2-5|            [Dahl, Findley, M...|
| 2016-1-1|            [Suchoff, Lowe, D...|
| 2018-2-6|            [Gottardo, Po, Go...|
| 2018-1-7|            [Macmahon, Hirsch...|
| 2016-6-7|            [Horah, Whitla, H...|
|2018-5-10|            [Gruber, Drago, S...|
| 2017-5-4|            [Srivastava, Ho-s...|
| 2018-4-7|            [Drago, Mayer, La...|
|2018-6-10|            [Badash, Decaro, ...|
| 2017-3-3|            [Badash, Marinko,...|
| 2017-3-5|            [Rosenbloom, Bals...|
| 2017-6-4|            [Whitla, Egan, Lo...|
| 2018-6-3|            [Cerda, Berenguie...|
| 2018-1-6|            [Zapata, Miao, Ne...|
| 2016-4-9|            [Badash, Dahlsted...|
| 2016-1-5|            [Suchoff, Srivast...|
|2018-1-10|            [Srivastava, Domi...|
| 2017-1-5|            [Dower, Zapata, M...|
| 2018-1-4

In [ ]:
%%sql
select "date", collect_list(customer.last_name) from orders group by "date"

Now try it on yourself.

For every order in 2016-1-1, return the list of products that appeared in that order:

In [18]:
orders_df.filter(orders_df["date"] == "2016-1-1").select(explode("items").alias("i"), "date", "i.product", "order_id").groupBy("order_id").agg(collect_list("i.product")).show()

+--------+-----------------------+
|order_id|collect_list(i.product)|
+--------+-----------------------+
|    8484|   [headphones, whis...|
|   33209|   [notebook, the bo...|
|   84024|   [computer screen,...|
|   91703|   [notebook, stuffe...|
|   28555|   [kettle, computer...|
|    3120|   [whiskey bottle, ...|
|   48533|               [kettle]|
|   97472|   [toaster, the boo...|
|    1280|   [whiskey bottle, ...|
|   74743|           [mouse trap]|
|   85793|   [whiskey bottle, ...|
|   23754|   [the book, the bo...|
|   24308|   [fan, fan, whiske...|
|    9418|           [mouse trap]|
|   98787|   [kettle, stuffed ...|
|   35723|   [computer screen,...|
|   47083|   [mouse trap, fan,...|
|   58037|   [fan, kettle, mou...|
|   66103|           [headphones]|
|   94704|   [notebook, notebo...|
+--------+-----------------------+
only showing top 20 rows



For every product, return the set of dates at which it's purchased:

In [22]:
orders_df.select(explode("items").alias("i"), "i.product", "date").groupBy("product").agg(collect_list("date")).show()

+---------------+--------------------+
|        product|  collect_list(date)|
+---------------+--------------------+
|       the book|[2018-2-4, 2016-6...|
|     mouse trap|[2017-4-9, 2017-1...|
|computer screen|[2018-2-4, 2018-2...|
| whiskey bottle|[2018-2-4, 2016-1...|
|        toaster|[2018-4-8, 2018-6...|
| stuffed animal|[2018-2-4, 2016-6...|
|         kettle|[2018-2-4, 2016-4...|
|            fan|[2018-2-4, 2016-1...|
|     headphones|[2018-2-4, 2016-6...|
|       notebook|[2016-6-6, 2017-2...|
+---------------+--------------------+



One of the drawbacks of the <font face="courier">collect_set/collect_list</font> method is they only accept one column as the argument. Later we will see how we can create nestedness on pretty much everything after we get the hang of the mighty JSONiq.

In [26]:
path = "confusion-2014-03-02/confusion-2014-03-02.json"

In [27]:
dataset = spark.read.json(path).cache()

In [30]:
dataset.limit(1).show()

+--------------------+-------+----------+---------+--------------------+---------+
|             choices|country|      date|    guess|              sample|   target|
+--------------------+-------+----------+---------+--------------------+---------+
|[Maori, Mandarin,...|     AU|2013-08-19|Norwegian|48f9c924e0d98c959...|Norwegian|
+--------------------+-------+----------+---------+--------------------+---------+



In [43]:
dataset = dataset.limit(10000)

In [44]:
dataset.filter(dataset["guess"]== dataset["target"] ).filter(dataset["target"] == "Russian").count()

191

In [45]:
dataset.filter(dataset["guess"]== dataset["target"] ).filter(dataset["target"] == "Hindi").count()

54

In [47]:
dataset.select("target").distinct().count()

68

In [49]:
dataset.filter(dataset["target"] == dataset["guess"]).orderBy("target", "country", "date").limit(5).collect()

[Row(choices=['Albanian', 'Armenian', 'Farsi', 'Punjabi'], country='AU', date='2013-08-20', guess='Albanian', sample='00b85faa8b878a14f8781be334deb137', target='Albanian'),
 Row(choices=['Albanian', 'Assyrian', 'Hebrew', 'Spanish'], country='AU', date='2013-08-20', guess='Albanian', sample='13722ceed1eede7ba597ade9b4cb9807', target='Albanian'),
 Row(choices=['Albanian', 'Tagalog', 'Hindi', 'Urdu'], country='AU', date='2013-08-20', guess='Albanian', sample='00b85faa8b878a14f8781be334deb137', target='Albanian'),
 Row(choices=['Albanian', 'Tagalog', 'Khmer', 'Portuguese'], country='AU', date='2013-08-20', guess='Albanian', sample='13722ceed1eede7ba597ade9b4cb9807', target='Albanian'),
 Row(choices=['Albanian', 'Arabic', 'Assyrian', 'Indonesian'], country='AU', date='2013-08-20', guess='Albanian', sample='efcd813daec1c836d9f030b30caa07ce', target='Albanian')]

In [55]:
dataset.groupBy("country", "target").count().filter(dataset["country"] == "NL").filter(dataset["target"] == "Maltese").show()

+-------+------+-----+
|country|target|count|
+-------+------+-----+
+-------+------+-----+



In [56]:
correct = dataset.filter(dataset["target"] == dataset["guess"]).count()

In [57]:
correct_first = dataset.filter(dataset["target"] == dataset["guess"]).filter(dataset["target"] == dataset["choices"][1]).count()

In [58]:
correct_first/correct

0.34119880661784646

In [69]:
data1 = dataset.rdd.map(lambda x: (x["target"], x["target"] == x["guess"])).groupByKey()

In [75]:
data2 = data1.mapValues(lambda x: list(x).count(True)/len(x)).sortBy(lambda x: x[1], ascending = False).collect()

In [76]:
data2

[('Italian', 0.9609756097560975),
 ('German', 0.944954128440367),
 ('French', 0.9420289855072463),
 ('Spanish', 0.92),
 ('Japanese', 0.9090909090909091),
 ('Cantonese', 0.9065934065934066),
 ('Korean', 0.9023255813953488),
 ('Arabic', 0.8973214285714286),
 ('Mandarin', 0.8970588235294118),
 ('Russian', 0.8925233644859814),
 ('Hebrew', 0.8846153846153846),
 ('Vietnamese', 0.8594594594594595),
 ('Polish', 0.8394160583941606),
 ('Czech', 0.8231292517006803),
 ('Thai', 0.8076923076923077),
 ('Portuguese', 0.8041237113402062),
 ('Ukrainian', 0.8033707865168539),
 ('Norwegian', 0.8028846153846154),
 ('Romanian', 0.8020833333333334),
 ('Swedish', 0.8009950248756219),
 ('Slovak', 0.7965116279069767),
 ('Greek', 0.7906976744186046),
 ('Danish', 0.7428571428571429),
 ('Swahili', 0.7423312883435583),
 ('Dutch', 0.7384615384615385),
 ('Lao', 0.7352941176470589),
 ('Slovenian', 0.7348484848484849),
 ('Finnish', 0.7323943661971831),
 ('Tamil', 0.7310344827586207),
 ('Yiddish', 0.7310344827586207),
 

# Exam Questions

In [71]:
movies_df = movies_df.select("actors", "genres", "name", movies_df["votes"].cast(IntegerType()), movies_df["year"].cast(IntegerType()), movies_df["rating"].cast(DoubleType()))

In [74]:
movies_df.filter(movies_df["year"] == "1991").count()

234

In [81]:
movies_df.select("name", "rating", "votes").orderBy("rating", "votes",ascending = False).collect()[0]

Row(name='Till Time Shall End', rating=10.0, votes=7)

In [88]:
movieage = movies_df.filter(movies_df["name"] == "Mob Town").select(explode("actors").alias("i"), "i.birth", "year")

In [99]:
movieageint = movieage.select("year", movieage["birth"].cast(IntegerType()))

In [104]:
movieageint.withColumn("age",movieageint["year"] - movieageint["birth"]).select(avg("age")).show()

+--------+
|avg(age)|
+--------+
|   23.75|
+--------+



Actor in the most movies 

In [106]:
movies_df.select(explode("actors").alias("i"), "i.name").select("name").distinct().count()

22737

In [108]:
movies_df.groupBy("year").sum("votes").orderBy("sum(votes)", ascending = False).show()

+----+----------+
|year|sum(votes)|
+----+----------+
|1957|   1673009|
|1954|   1586085|
|1960|   1539885|
|1959|   1525685|
|1962|   1372253|
|1968|   1239030|
|1939|   1165867|
|1964|   1138673|
|1955|   1128751|
|1958|   1094661|
|1941|   1081774|
|1953|   1081183|
|1963|   1074507|
|1950|   1043050|
|1946|   1014856|
|1961|    998506|
|1940|    978235|
|1942|    972899|
|1951|    931534|
|1956|    872253|
+----+----------+
only showing top 20 rows



In [112]:
movies_df.select(explode("actors").alias('i'), "i.name", "rating").groupBy("i.name").avg("rating").orderBy("avg(rating)", ascending = False).show()

+--------------------+-----------------+
|                name|      avg(rating)|
+--------------------+-----------------+
|     Walter Kiaulehn|             10.0|
|      Charles Cullum|             10.0|
|     Willem Holsboer|             10.0|
|       John H. Moore|             10.0|
|    Kurt Wehofschitz|              9.9|
|       Ottokar Runze|              9.8|
|    Erwin Scherschel|              9.8|
|         John Wesley|              9.7|
|        Eugen Verber|              9.7|
|        Paul Günther|              9.6|
|      Walter Tarrach|              9.6|
|         Otto Collin|              9.6|
|Hans Müller-Weste...|              9.6|
|     Herbert Johnson|              9.6|
|       Allen Sferios|              9.6|
|          Karl Luley|              9.6|
|   Friedrich Siemers|              9.6|
|     Bernd M. Bausch|              9.6|
|      Wolfgang Spier|             9.55|
|     Beppo Schwaiger|9.533333333333333|
+--------------------+-----------------+
only showing top

In [114]:
movies_df.select(explode("actors").alias("i"), "i.name").select("name").distinct().count()

22737

In [115]:
explodedactors= movies_df.select(explode("actors").alias("i"), "i.name")

In [117]:
explodedactors.filter(explodedactors["name"] == "Mickey Rooney").count()

118

In [118]:
explodedactorsR= movies_df.select(explode("actors").alias("i"), "i.name", "rating")

In [122]:
explodedactorsR.groupBy("i.name").avg("rating").orderBy("avg(rating)", ascending = False).show()

+--------------------+-----------------+
|                name|      avg(rating)|
+--------------------+-----------------+
|      Charles Cullum|             10.0|
|     Walter Kiaulehn|             10.0|
|     Willem Holsboer|             10.0|
|       John H. Moore|             10.0|
|    Kurt Wehofschitz|              9.9|
|    Erwin Scherschel|              9.8|
|       Ottokar Runze|              9.8|
|         John Wesley|              9.7|
|        Eugen Verber|              9.7|
|         Otto Collin|              9.6|
|      Walter Tarrach|              9.6|
|   Friedrich Siemers|              9.6|
|       Allen Sferios|              9.6|
|Hans Müller-Weste...|              9.6|
|     Herbert Johnson|              9.6|
|     Bernd M. Bausch|              9.6|
|          Karl Luley|              9.6|
|        Paul Günther|              9.6|
|      Wolfgang Spier|             9.55|
|     Beppo Schwaiger|9.533333333333333|
+--------------------+-----------------+
only showing top

In [123]:
intrating = movies_df.select("actors", "genres", "name", movies_df["votes"].cast(IntegerType()), movies_df["year"].cast(IntegerType()), movies_df["rating"].cast(IntegerType()))

In [124]:
intrating.groupBy("rating").avg("votes").orderBy("avg(votes)").show()

+------+------------------+
|rating|        avg(votes)|
+------+------------------+
|    10|               5.5|
|     4|124.35920889987639|
|     3|154.43396226415095|
|     9|165.26829268292684|
|     2|  168.562874251497|
|     5|176.38442840306422|
|     6|294.60998965794613|
|     1|            449.72|
|     7| 763.4944402324994|
|     8|1788.9719594594594|
+------+------------------+



In [126]:
sverge

6.673361642793781

In [148]:
Ravge = movies_df.select(avg("rating")).collect()[0][0]

In [149]:
countbelow= movies_df.filter(movies_df["rating"] > Ravge).groupby("genres").count().withColumnRenamed("count", "below")

In [150]:
countOverall = movies_df.groupby("genres").count()

In [151]:
bigtable = countOverall.join(countbelow.alias("i"), "genres")

In [152]:
bigtable.withColumn("ratio", bigtable["below"]/bigtable["count"]).orderBy("ratio").show()

+--------------------+-----+-----+-------------------+
|              genres|count|below|              ratio|
+--------------------+-----+-----+-------------------+
|Comedy,Horror,Sci-Fi|   18|    1|0.05555555555555555|
|       Family,Sci-Fi|   32|    2|             0.0625|
|Animation,Music,S...|   31|    2|0.06451612903225806|
| Drama,Short,Western|   15|    1|0.06666666666666667|
| Drama,History,Short|   27|    2|0.07407407407407407|
|     Musical,Western|   13|    1|0.07692307692307693|
| Action,Drama,Horror|   12|    1|0.08333333333333333|
|Adventure,Fantasy...|   12|    1|0.08333333333333333|
|Horror,Sci-Fi,Thr...|   22|    2|0.09090909090909091|
|Action,Music,Romance|   22|    2|0.09090909090909091|
|       Romance,Short|   42|    4|0.09523809523809523|
|Adventure,Fantasy...|   10|    1|                0.1|
|  Adventure,Thriller|   10|    1|                0.1|
|     Adventure,Short|   20|    2|                0.1|
|Action,Adventure,...|  117|   12|0.10256410256410256|
|       Fa